In [14]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model
from sklearn import datasets
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from yellowbrick.target import FeatureCorrelation
from sklearn.model_selection import ShuffleSplit
from sklearn.tree import DecisionTreeRegressor
import seaborn as sns
from datetime import datetime

In [36]:
df_1 = pd.read_csv(".\\audiobooks.csv")
df_2 = pd.read_csv(".\\audiobooks_2.csv")
df = df_1.merge(df_2, on='audiobook_id')
df = df[['price', 'release_date', 'length', 'rating_overall', 'rating_performance', 'rating_story']].dropna()

def convert_time(time_str):
    year = time_str[-2:]
    time_str = time_str[:-2] + '20' + year
    date_time = datetime.strptime(time_str, '%m-%d-%Y')
    return date_time.timestamp()

def convert_duration(length):
    # 9 hrs and 15 mins
    
    if 'hrs' in length:
        hours = length.split('hrs')[0].strip() # 9
    else:
        hours = length.split('hr')[0].strip() # 9
    if 'mins' in length:
        minutes = length[:-5].split(' ')[-1] # 15
        time = int(hours)*60 + int(minutes)
    else:
        time = int(hours)*60
    return time

df['release_date'] = df['release_date'].apply(convert_time)
df['length'] = df['length'].apply(convert_duration)
df['price'] = df['price'].apply(lambda s : float(s[1:]))
df

,price,release_date,length,rating_overall,rating_performance,rating_story
0,25.51,1.667257e+09,529,4.6,4.7,4.6
1,23.62,1.666044e+09,521,4.7,4.8,4.6
2,26.90,1.662415e+09,1446,4.8,4.9,4.7
3,13.99,1.659996e+09,386,4.9,4.9,4.9
4,35.00,1.666044e+09,1042,4.6,4.7,4.5
...,...,...,...,...,...,...
1158,59.99,1.654553e+09,1220,4.9,5.0,4.8
1159,23.90,1.605568e+09,487,4.6,4.6,4.6
1160,24.90,1.569276e+09,593,4.6,4.9,4.6
1161,17.92,1.600121e+09,591,4.4,4.6,4.3


Določimo s katerimi featurji bomo delali.

In [55]:
X, y = df.drop(["price"], axis=1), df["price"]
selector = SelectFromModel(estimator=RandomForestRegressor()).fit(X, y)
selector.get_feature_names_out()

array(['release_date', 'length'], dtype=object)

Uporabili bomo random forest regressor od sklearn.

In [56]:
ss = ShuffleSplit(n_splits=5, random_state=0)
def average_score(scores, n):
    ''' average in list, only last n values '''
    s=0
    for i in range(n):
        s+=scores[-(i+1)]
    return s/n
#X = df[['length', 'release_date']]
def RandomForestRegression():
    model = RandomForestRegressor(max_depth=4, random_state=0)
    
    Scores = []
    for train_index, test_index in ss.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train, y_train)
        
        score = model.score(X_test, y_test)
        Scores.append(score)
    return Scores

In [57]:
RandomForestRegression()

[0.4037820028063538,
 0.5277845618139929,
 0.5238792349261208,
 0.5292778690035328,
 0.4911498221395837]